In [8]:
const ○ = 1 # 𝟘 ○ OFF
const ● = 2 # 𝟙 ● ON

2

In [9]:
state_count = 4

# state by state flat 
p = 
Float64[
#              ○ ● ○ ● ○ ● ○ ● ○ ● ○ ● ○ ● ○ ●
#              ○ ○ ● ● ○ ○ ● ● ○ ○ ● ● ○ ○ ● ●
#              ○ ○ ○ ○ ● ● ● ● ○ ○ ○ ○ ● ● ● ●
#              ○ ○ ○ ○ ○ ○ ○ ○ ● ● ● ● ● ● ● ●    
#= ○ ○ ○ ○ =#  1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ● ○ ○ ○ =#  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ● ○ ○ =#  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0;
#= ● ● ○ ○ =#  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ○ ● ○ =#  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ● ○ ● ○ =#  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0;
#= ○ ● ● ○ =#  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0;
#= ● ● ● ○ =#  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ○ ○ ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ● ○ ○ ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ○ ● ○ ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ● ● ○ ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ○ ○ ● ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ● ○ ● ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1;
#= ○ ● ● ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ● ● ● ● =#  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0;
]
q = reshape(p, (ntuple(x->2, state_count*2)));
length(size(q))

8

In [53]:
mutable struct TPM
    tpm
    elements # vector of elements
    next_state_elements_index # index in vector where the next_state elements start
end


function make_tpm(flat, elements)
    @assert size(flat) == (length(elements)^2, length(elements)^2)
    
    TPM(
        reshape(flat, (ntuple(x->2, length(elements)*2))),
        [elements..., elements...],
        length(elements)+1
    )
end


function tpm_element_dim(tpm, element, current)
    current ? element_dim(tpm.elements, element) : element_dim(tpm.elements, element, tpm.next_state_elements_index)
end

function element_dim(elements, element, start_at)
    findnext(==(element), elements, start_at)
end

function remove_element(elements, element, start_at=1)
    deleteat!(copy(elements), element_dim(elements, element, start_at))
end
    
    

function current_state_count(tpm)
    tpm.next_state_elements_index-1
end

function next_state_count(tpm)
    length(tpm.elements) - tpm.next_state_elements_index + 1
end
    
    
function flatten_tpm(tpm)
    println(current_state_count(tpm), " ", next_state_count(tpm))
    reshape(tpm.tpm, 2^current_state_count(tpm), 2^next_state_count(tpm)) 
end

    
function fix_condition(tpm, element, current, value) 
    @assert value == ○ || value == ●
    
    len = length(size(tpm.tpm))
    
    dim = tpm_element_dim(tpm, element, current)
    current_count = tpm.next_state_elements_index - 1
    
    tpm2 = tpm.tpm[ntuple(x->Colon(), dim-1)..., value, ntuple(x->Colon(), len-dim)...]
    tpm3 = tpm2 ./ sum(tpm2, dims=(current ? [current_count:len-1...] : [current_count+1:len-1...])) # normailizing rows

    elements = remove_element(tpm.elements, element, current ? 1 : tpm.next_state_elements_index)
    next_state_elements_index = current ? (tpm.next_state_elements_index - 1) : tpm.next_state_elements_index
    
    TPM(tpm3, elements, next_state_elements_index)
end 

function marginalize(tpm, element, current) 
    println("marginalize", size(tpm.tpm), element, current ? "Current" : "Next")
    len = length(size(tpm.tpm))
    
    dim = tpm_element_dim(tpm, element, current)
        
    current_count = tpm.next_state_elements_index - 1
   
    tpm2 = tpm.tpm[ntuple(x->Colon(), dim-1)..., ○, ntuple(x->Colon(), len-dim)...] + tpm.tpm[ntuple(x->Colon(), dim-1)..., ●, ntuple(x->Colon(), len-dim)...]
    tpm3 = tpm2 ./ sum(tpm2, dims=(current ? [current_count:len-1...] : [current_count+1:len-1...])) # normailizing rows
    
    elements = remove_element(tpm.elements, element, current ? 1 : tpm.next_state_elements_index)
    next_state_elements_index = current ? (tpm.next_state_elements_index - 1) : tpm.next_state_elements_index
    
    TPM(tpm3, elements, next_state_elements_index)
end 

marginalize (generic function with 1 method)

In [56]:
struct System
    elements
    states
    element_states
end

function make_system(elements, states)
    System(elements, states, Dict([(elements[index], states[index]) for (index, element) in enumerate(elements)]))
end


struct CandidateMechanism
    elements
    mechanism_elements
    perview_elements
end

function system_elements(system)
    keys(system.element_states)
end

function external_elements(system, mechanism)
    setdiff(system.elements, mechanism.elements)
end

function fix_elements(tpm, system, elements)
    for element in elements
        tpm = fix_condition(tpm, element, true, system.element_states[element])
        tpm = marginalize(tpm, element, false) 
    end
    tpm
end





# Cause Repertoire
# Marganilize out non perview elements
# Marganilize out non mechanism elements

# Effect Repertoire
# Marganilize out non perview elements
# Marganilize out non mechanism elements
    
function repertoire(tpm, system, mechanism, cause)
    # effect repertoire these are t+1
    # cause repertoire these are t
    for element in setdiff(mechanism.elements, mechanism.perview_elements)
        tpm = marginalize(tpm, element, cause)
    end
    
    # effect repertoire these are t
    # cause repertoire these are t+1
    for element in setdiff(mechanism.elements, mechanism.mechanism_elements)
        tpm = marginalize(tpm, element, !cause)
    end
  
 
    
    if !cause
        to_fix = intersect(mechanism.mechanism_elements, tpm.elements[tpm.next_state_elements_index:length(tpm.elements)])
        for f in to_fix
            tpm = fix_condition(tpm, f, true, system.element_states[f]) 
        end
    else
        to_fix = intersect(mechanism.mechanism_elements, tpm.elements[1:(tpm.next_state_elements_index-1)])
        for f in to_fix
            tpm = fix_condition(tpm, f, false, system.element_states[f]) 
        end
    end

    tpm
end

function cause_repertoire(tpm, system, mechanism)
    repertoire(tpm, system, mechanism, true)
end

function effect_repertoire(tpm, system, mechanism)
    repertoire(tpm, system, mechanism, false)
end


system = make_system([:a, :b, :c, :d], [●, ○, ○, ○])

println(system)

tpm = make_tpm(p, system.elements)

println(tpm)

mechanism = CandidateMechanism([:a, :b, :c], [:c], [:b, :c])

tpm = fix_elements(tpm, system, external_elements(system, mechanism))

println(flatten_tpm(tpm))

println("cause_repertoire")
tpm_cause = cause_repertoire(tpm, system, mechanism)
println("effect_repertoire")
tpm_effect = effect_repertoire(tpm, system, mechanism)

println(tpm_cause)
println(tpm_effect)

println(flatten_tpm(tpm_cause))
println(flatten_tpm(tpm_effect))

# tpm_a = fix_condition(tpm_a, :d, true, ○)
# tpm_a = marginalize(tpm_a, :d, false)

# tpm_b = marginalize(tpm_a, :b, false)
# tpm_b = marginalize(tpm_b, :c, false)
# tpm_b = marginalize(tpm_b, :a, true)
# tpm_b = marginalize(tpm_b, :b, true)
# tpm_b = marginalize(tpm_b, :c, true)

# tpm_c = marginalize(tpm_a, :a, true)
# tpm_c = marginalize(tpm_c, :b, true)

# tpm_d = marginalize(tpm_c, :a, false)
# tpm_d = fix_condition(tpm_d, :c, true, ○)

# tpm_e = marginalize(tpm_c, :b, false)
# tpm_e = fix_condition(tpm_e, :c, true, ○)

# println(flatten_tpm(tpm_d))
# println(flatten_tpm(tpm_e))

System([:a, :b, :c, :d], [2, 1, 1, 1], Dict(:a => 2, :b => 1, :d => 1, :c => 1))
TPM([1.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;; 0.0 0.0; 0.0 1.0;;; 1.0 0.0; 0.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 1.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;;;; 0.0 0.0; 1.0 0.0;;; 0.0 0.0; 0.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;; 0.0 1.0; 0.0 0.0;;; 0.0 1.0; 0.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 1.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;; 1.0 0.0; 0.0 1.0;;; 1.0 0.0; 0.0 0.0;;;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.0;;;; 0.0 0.0; 0.0 0.0;;; 0.0 0.0; 0.0 0.

In [45]:
tpm_cause

TPM([0.5 0.5; 0.5 0.5], [:b, :c], 3)

In [42]:
flatten_tpm(tpm_cause)

2 0


4×1 Matrix{Float64}:
 0.5
 0.5
 0.5
 0.5